# SparkFund

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [8]:
rounds = pd.read_csv("Datasets/rounds2.csv",encoding='iso-8859-1 ')
companies = pd.read_csv("../input/spark-fund-investment-analysis/datasets/companies.txt",sep='\t',encoding='iso-8859-1')

AttributeError: 'DataFrame' object has no attribute 'permalink'